# Clasificación de enfermedades en hojas de maíz

Hemos utilizado la base de datos PlantVillage [1] elaborada por Hughes et al. la cual consiste en alrededor de 87,000 saludables y no saludables imagenes de hojas de plantas dividida en 38 categorias por especie y enfermedad. Se utiliza la técnica de "transfer learning" para la red MobileNet, además se utilizan los pesos de un pre-entrenamiento utilizando ImageNet.

* ![PlantVillage Dataset Samples](https://i.imgur.com/Zcxdrlc.png)
Figure 1. Ejemplos de la base de datos PlantVillage 

## Referencias

[1] Hughes, David P., and Marcel Salathe. “An Open Access Repository of Images on Plant Health to Enable the Development of Mobile Disease Diagnostics.” ArXiv:1511.08060 [Cs], Apr. 2016. arXiv.org, http://arxiv.org/abs/1511.08060.


## Configuración Inicial

In [ ]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np

import os

In [ ]:
image_size = 224
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)

batch_size = 32
epochs = 25

## Obtención de datos

A fin de aprovechar al máximo las muestras de entrenamiento de la base de datos, procedemos a realizar un aumento de los mismos a través de una serie de transformaciones aleatorias, de modo que el modelo MobileNet propuesto nunca ve la misma imagen dos veces. Esto ayuda a evitar el sobreajuste y ayuda a generalizar mejor el modelo.

En TensorFlow, esto se puede hacer a través de la clase `tf.keras.preprocessing.image.ImageDataGenerator`. Esta clase permite:

- configurar las transformaciones aleatorias y operaciones de normalización que se realizarán en las imagenes durante el entrenamiento.
- crear instancias de generadores de lotes (batchs) de imágenes aumentadas (y sus etiquetas) a través de `.flow(datos, etiquetas)` o `.flow_from_directory(directory)`. Estos generadores se pueden usar con los métodos del modelo `tf.keras` que aceptan generadores de datos como entradas, `ajuste (fit)`, `evaluar (evaluate)` y `predecir (predecir)`.

In [ ]:
base_dir = "../input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"valid")

Hemos realizado los siguientes aumentos a las imágenes:

- `width_shift` y `height_shift` son rangos (como una fracción del ancho o alto total) dentro de los cuales se pueden trasladar aleatoriamente imágenes vertical u horizontalmente.
- `rescale` es un valor por el que multiplicaremos los datos antes de cualquier otro procesamiento. Las imágenes originales consisten en coeficientes RGB en el rango de 0-255, pero tales valores serían demasiado altos para que los modelos los procesen, por lo que apuntamos a valores entre 0 y 1 escalando con un factor 1/255. 
- `shear_range` permite aplicar aleatoriamente transformaciones de corte
- `zoom_range` permite hacer zoom al azar dentro de las imágenes
- `fill_mode` es la estrategia utilizada para rellenar píxeles recién creados, que pueden aparecer después de una rotación o un cambio de ancho/alto.


In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0,
                                                             shear_range = 0.2,
                                                             zoom_range = 0.2,
                                                             width_shift_range = 0.2,
                                                             height_shift_range = 0.2,
                                                             fill_mode="nearest")

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)

Posteriormente, preparamos los datos para lo cual usamos `.flow_from_directory()` para generar lotes (batches) de datos de imagen (y sus etiquetas) directamente desde las imágenes.


In [ ]:
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size = (image_size, image_size),
                                               batch_size = batch_size,
                                               class_mode = "categorical")

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = (image_size, image_size),
                                             batch_size = batch_size,
                                             class_mode = "categorical")

## Crear un archivo con las clases

Tiene por objetivo saber a qué clase corresponde y a qué especie y enfermedad, por lo que creamos un archivo `json` que muestra las etiquetas correspondientes y los índices de clase.


In [ ]:
categories = list(train_data.class_indices.keys())
print(train_data.class_indices)

In [ ]:
import json
with open('class_indices.json','w') as f:
  json.dump(train_data.class_indices, f)

from IPython.display import FileLink
FileLink(r'class_indices.json')

## Entrenamiento (Training)

Primeramente, obtenemos el modelo base de MobileNet sin incluir las capas superiores, ya que queremos usarlo para 4 clases, que representan 3 enfermedades (Cercospora Gray Leaf Spot, Common Rust y  Northern Leaf Blight) y la clase de hojas saludable (Healthy-corn) para las hojas de maíz. Posteriormente, usamos los pesos pre-entrenados de ImageNet.



In [ ]:
base_model = tf.keras.applications.MobileNet(weights = "imagenet",
                                             include_top = False,
                                             input_shape = input_shape)

base_model.trainable = False

En forma seguida, creamos un pequeño modelo ascendente sobre MobileNet

In [ ]:
inputs = keras.Input(shape = input_shape)

x = base_model(inputs, training = False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(categories), 
                          activation="softmax")(x)

model = keras.Model(inputs = inputs, 
                    outputs = x, 
                    name="LeafDisease_MobileNet")

En los experimento realizados, determinamos que el optimizador de Adam funciona muy bien con tasa de aprendizaje = 0.001, los valores $\beta_1 = 0.9$, $\beta_2=0.999$ y $\epsilon=1^{-8}$ 

In [ ]:
optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer = optimizer,
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
              metrics=[keras.metrics.CategoricalAccuracy(), 
                       'accuracy'])

In [ ]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=epochs,
                    steps_per_epoch=100,
                    validation_steps=100)

## Validando el proceso de entrenamiento

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

fig = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training")
plt.plot(epochs,val_loss,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

epochs = range(len(acc))

fig = plt.figure(figsize=(10,6))
plt.plot(epochs,acc,c="red",label="Training")
plt.plot(epochs,val_acc,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

## Guardando el model entrenado

Finalmente, guardamos el modelo en el formato estándar TensorFlow 2 SavedModel.

In [ ]:
model.save('plant_disease')